In [1]:
import numpy as np
import copy

period_table = {1:'H',2:'He',3:'Li',4:'Be',5:'B',6:'C',7:'N',8:'O',9:'F',10:'Ne',11:'Na',12:'Mg',13:'Al',14:'Si',15:'P',16:'S',17:'Cl',18:'Ar',
                19:'K',20:'Ca',21:'Sc',22:'Ti',23:'V',24:'Cr',25:'Mn',26:'Fe',27:'Co',28:'Ni',29:'Cu',30:'Zn',31:'Ga',32:'Ge',33:'As',34:'Se',35:'Br',36:'Kr',
                37:'Rb',38:'Sr',39:'Y',40:'Zr',41:'Nb',42:'Mo',43:'Tc',44:'Ru',45:'Rh',46:'Pd',47:'Ag',48:'Cd',49:'In',50:'Sn',51:'Sb',52:'Te',53:'I',54:'Xe',
                55:'Cs',56:'Ba',57:'La',58:'Ce',59:'Pr',60:'Nd',61:'Pm',62:'Sm',63:'Eu',64:'Gd',65:'Tb',66:'Dy',67:'Ho',68:'Er',69:'Tm',70:'Yb',71:'Lu',
                72:'Hf',73:'Ta',74:'W',75:'Re',76:'Os',77:'Ir',78:'Pt',79:'Au',80:'Hg',81:'Tl',82:'Pb',83:'Bi',84:'Po',85:'At',86:'Rn'}

In [2]:
class AtomicParamTable():

    def __init__(self):
        self.head=r"""
% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & $\eta_A$& $\Gamma_A \ast 10$& $\alpha_A$ & $Y_A^{\text{eff}}$ & $f_{\text{XC}}^{\mu_A} \ast 100$& $f_{\text{XC}}^{\Theta_A} \ast 100$\\ \midrule
"""
        self.tail=r""" \bottomrule
\end{tabular}
\end{table}
"""
        self.table=copy.deepcopy(self.head)
        self.__finished=False

        
    def add_line(self,elem,gam,gam3,repa,repb,dpol,qpol):
        assert self.__finished==False, "You can't add a line because the Table was set before this action, correct your code"
        line=f"{elem: <6} & {gam: .6f} & {gam3: .6f} & {repa: .6f} & {repb: .6f} & {dpol: .6f} & {qpol: .6f} \\\\"
        self.table+="\n"+ line

    def set_table(self):
        assert self.__finished==False, "Table was set before this action, correct your code"
        self.table+=self.tail
        self.__finished=True
        
    def get_table(self):
        if self.__finished:
            return self.table
        else:
            self.set_table()
            return self.table
            

class ShellTable():

    def __init__(self):
        self.head=r"""
% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & level & $k_{A,l}^{\text{poly}}  \ast 100$& $\kappa_A^l \ast 10$& $H_{\text{CN}_A'}^l (eV) *10$& $H_A^l( eV)$& $\zeta_l$ \\ \midrule
"""
        self.tail=r""" \bottomrule
\end{tabular}
\end{table}
"""
        self.table=copy.deepcopy(self.head)
        self.__finished=False

        
    def add_line(self,elem,orb,poly,lpar,kcn,lev,exp):
        assert self.__finished==False, "You can't add a line because the Table was set before this action, correct your code"
        line=f"{elem: <6} & {orb: <6} & {poly: .6f} & {lpar: .6f} & {kcn: .6f} & {lev: .6f} & {exp: .6f} \\\\"
        self.table+="\n"+ line

    def set_table(self):
        assert self.__finished==False, "Table was set before this action, correct your code"
        self.table+=self.tail
        self.__finished=True
        
    def get_table(self):
        if self.__finished:
            return self.table
        else:
            self.set_table()
            return self.table
        

In [3]:
class ElemParams():
    def __init__(self,elem_lines):
        self.lines=elem_lines
        self.Z=0
        self.GAM=None
        self.GAM3=None
        self.DPOL=None
        self.QPOL=None
        self.REPA=None
        self.REPB=None
        self.shell={}
        self.orbitals=None
        self.read_lines()
        self.APList=[]
        self.SPTable=[]
        self.set_table_data()

    def set_table_data(self):
        self.APList=[period_table[self.Z], self.GAM, self.GAM3, self.REPA, self.REPB, self.DPOL, self.QPOL]
        
        for ao,lev,exp in zip(self.orbitals,self.shell["lev"],self.shell["exp"]):
            if ao[1:]=="s":
                self.SPTable.append([period_table[self.Z], ao, self.shell["POLYS"], np.float32(0.0), self.shell["KCNS"],lev,exp])
            elif ao[1:]=="p":
                if not "LPARP" in self.shell:
                    self.shell["LPARP"]=np.float32(0.0)
                if not "KCNP" in self.shell:
                    self.shell["KCNP"]=np.float32(0.0)
                    
                self.SPTable.append([period_table[self.Z], ao, self.shell["POLYP"], self.shell["LPARP"], self.shell["KCNP"],lev,exp])
            elif ao[1:]=="d":
                self.SPTable.append([period_table[self.Z], ao, self.shell["POLYD"], self.shell["LPARD"], self.shell["KCND"],lev,exp])
                       
    def read_lines(self):
        self.Z=int(self.lines[0][:6].split('=')[1])
        label,orb=self.lines[1].strip().split("=")
        assert label=="ao", "Error parsing Shell information"
        self.orbitals=[orb[i:i+2] for i in range(0, len(orb), 2)]
        self.shell={}
        
        for line in self.lines[2:]:
            if line.strip()=="$end":
                break
            label,value=line.strip().split("=")
            if label.strip()=="lev":
                self.shell["lev"]=np.fromstring(value, dtype=np.float32, sep=' ')
            elif label.strip()=="exp":
                self.shell["exp"]=np.fromstring(value, dtype=np.float32, sep=' ')          
            elif label.strip()=="GAM":
                self.GAM=np.float32(value)
            elif label.strip()=="GAM3":
                self.GAM3=np.float32(value)  
            elif label.strip()=="KCNS":
                self.shell["KCNS"]=np.float32(value)
            elif label.strip()=="KCNP":
                self.shell["KCNP"]=np.float32(value)
            elif label.strip()=="KCND":
                self.shell["KCND"]=np.float32(value)
            elif label.strip()=="DPOL":
                self.DPOL=np.float32(value)
            elif label.strip()=="QPOL":
                self.QPOL=np.float32(value)
            elif label.strip()=="REPA":
                self.REPA=np.float32(value)
            elif label.strip()=="REPB":
                self.REPB=np.float32(value)
            elif label.strip()=="POLYS":
                self.shell["POLYS"]=np.float32(value)
            elif label.strip()=="POLYP":
                self.shell["POLYP"]=np.float32(value)
            elif label.strip()=="POLYD":
                self.shell["POLYD"]=np.float32(value)
            elif label.strip()=="LPARP":
                self.shell["LPARP"]=np.float32(value)
            elif label.strip()=="LPARD":
                self.shell["LPARD"]=np.float32(value) 
                        

In [4]:
class GFN2xTB_Param():

    def __init__(self,param_file: str):
        self.param_lines=open(param_file,'r').readlines()
        self.info=''
        self.glb_ln=[]
        self.pairpar_ln=[]
        self.elem_lines={}
        self.read_param_file(self.param_lines)
        self.eparams={}
        self.parse_elements()
        
    def read_param_file(self, lines):
        info_flag=True
        globpar_flag=False
        pairpar_flag=False
        elem_flag=False
        z=0
        
        for ln in lines:
            if info_flag:
                if "$globpar" in  ln:
                    info_flag=False
                    globpar_flag=True
                    self.glb_ln.append(ln)
                else:
                    self.info+=ln
                
            elif globpar_flag:
                self.glb_ln.append(ln)
                if "$end" in ln:
                    globpar_flag=False
                    pairpar_flag=True
            
            elif pairpar_flag:
                self.pairpar_ln.append(ln)
                if "$end" in ln:
                    pairpar_flag=False
                    elem_flag=True
            
            elif elem_flag:
                if ln.startswith("$Z=") and z==0:
                    z=int(ln[:6].split('=')[1])
                    self.elem_lines[z]=[]
                    self.elem_lines[z].append(ln)
                elif "$end" in ln:
                    self.elem_lines[z].append(ln)
                    z=0          
                elif z > 0:
                    self.elem_lines[z].append(ln)
    
    def parse_elements(self):
        for z in self.elem_lines.keys():
            self.eparams[z]=ElemParams(self.elem_lines[z])


In [5]:
def print_atm_parms(param, elem_list):
    atm_prm=AtomicParamTable()
    for i in elem_list:
        atm_prm.add_line(*param.eparams[i].APList)
    atm_prm.set_table()
    print(atm_prm.get_table())

def print_shell_parms(param, elem_list):
    shell_tbl=ShellTable()
    for i in elem_list:
        for line in param.eparams[i].SPTable:
            shell_tbl.add_line(*line)
    shell_tbl.set_table()
    print(shell_tbl.get_table())
    

# GFN2-xTB's parametrizations 

## CCR

In [6]:
param=GFN2xTB_Param(param_file='CCR/param_gfn2-xtb.txt')
orig=GFN2xTB_Param(param_file='/home/bin/anaconda3/envs/xtbfitter/share/xtb/param_gfn2-xtb.txt')
elems=[1,6,7,8,16]

In [7]:
print_atm_parms(param=orig, elem_list=elems)
print_atm_parms(param=param, elem_list=elems)


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & $\eta_A$& $\Gamma_A \ast 10$& $\alpha_A$ & $Y_A^{\text{eff}}$ & $f_{\text{XC}}^{\mu_A} \ast 100$& $f_{\text{XC}}^{\Theta_A} \ast 100$\\ \midrule

H      &  0.405771 &  0.800000 &  2.213717 &  1.105388 &  5.563889 &  0.027431 \\
C      &  0.538015 &  1.500000 &  1.247655 &  4.231078 & -0.411674 &  0.213583 \\
N      &  0.461493 & -0.639780 &  1.682689 &  5.242592 &  3.521273 &  2.026786 \\
O      &  0.451896 & -0.517134 &  2.165712 &  5.784415 & -4.935670 & -0.310828 \\
S      &  0.339971 & -0.501722 &  1.214553 &  14.995090 & -0.151117 &  0.442859 \\ \bottomrule
\end{tabular}
\end{table}


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element 

In [8]:
print_shell_parms(param=orig, elem_list=elems)
print_shell_parms(param=param, elem_list=elems)


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & level & $k_{A,l}^{\text{poly}}  \ast 100$& $\kappa_A^l \ast 10$& $H_{\text{CN}_A'}^l (eV) *10$& $H_A^l( eV)$& $\zeta_l$ \\ \midrule

H      & 1s     & -0.953618 &  0.000000 & -0.500000 & -10.707211 &  1.230000 \\
C      & 2s     & -2.294321 &  0.000000 & -0.102144 & -13.970922 &  2.096432 \\
C      & 2p     & -0.271102 &  1.056358 &  0.161657 & -10.063292 &  1.800000 \\
N      & 2s     & -8.506003 &  0.000000 & -1.955336 & -16.686243 &  2.339881 \\
N      & 2p     & -2.504201 &  1.164892 &  0.561076 & -12.523956 &  2.014332 \\
O      & 2s     & -14.955291 &  0.000000 &  0.117826 & -20.229984 &  2.439742 \\
O      & 2p     & -3.350819 &  1.497020 & -0.145102 & -15.503117 &  2.137023 \\
S      & 3s     & -25.855520 &  0.000000 & -0.256951 & -20.029654 &  1.981333 \\
S      & 3p     & -8.0

## DHFR

In [9]:
param=GFN2xTB_Param(param_file='DHFR/param_gfn2-xtb.txt')
orig=GFN2xTB_Param(param_file='/home/bin/anaconda3/envs/xtbfitter/share/xtb/param_gfn2-xtb.txt')
elems=[1,6,7,8]

In [10]:
print_atm_parms(param=orig, elem_list=elems)
print_atm_parms(param=param, elem_list=elems)


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & $\eta_A$& $\Gamma_A \ast 10$& $\alpha_A$ & $Y_A^{\text{eff}}$ & $f_{\text{XC}}^{\mu_A} \ast 100$& $f_{\text{XC}}^{\Theta_A} \ast 100$\\ \midrule

H      &  0.405771 &  0.800000 &  2.213717 &  1.105388 &  5.563889 &  0.027431 \\
C      &  0.538015 &  1.500000 &  1.247655 &  4.231078 & -0.411674 &  0.213583 \\
N      &  0.461493 & -0.639780 &  1.682689 &  5.242592 &  3.521273 &  2.026786 \\
O      &  0.451896 & -0.517134 &  2.165712 &  5.784415 & -4.935670 & -0.310828 \\ \bottomrule
\end{tabular}
\end{table}


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & $\eta_A$& $\Gamma_A \ast 10$& $\alpha_A$ & $Y_A^{\text{eff}}$ & $f_{\text{XC}}^{\

In [11]:
print_shell_parms(param=orig, elem_list=elems)
print_shell_parms(param=param, elem_list=elems)


% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\caption{}
\label{tab:my-table}
\begin{tabular}{@{}|l|l|l|l|l|l|l|@{}}
\toprule
Element & level & $k_{A,l}^{\text{poly}}  \ast 100$& $\kappa_A^l \ast 10$& $H_{\text{CN}_A'}^l (eV) *10$& $H_A^l( eV)$& $\zeta_l$ \\ \midrule

H      & 1s     & -0.953618 &  0.000000 & -0.500000 & -10.707211 &  1.230000 \\
C      & 2s     & -2.294321 &  0.000000 & -0.102144 & -13.970922 &  2.096432 \\
C      & 2p     & -0.271102 &  1.056358 &  0.161657 & -10.063292 &  1.800000 \\
N      & 2s     & -8.506003 &  0.000000 & -1.955336 & -16.686243 &  2.339881 \\
N      & 2p     & -2.504201 &  1.164892 &  0.561076 & -12.523956 &  2.014332 \\
O      & 2s     & -14.955291 &  0.000000 &  0.117826 & -20.229984 &  2.439742 \\
O      & 2p     & -3.350819 &  1.497020 & -0.145102 & -15.503117 &  2.137023 \\ \bottomrule
\end{tabular}
\end{table}


% Please add the following required packages to your document p